In [1]:
import pandas as pd
import numpy as np
from numpy import log2 as log
import pprint

In [2]:
eps=np.finfo(float).eps

In [3]:
def find_entropy(df):
    cl=df.keys()[-1]
    entropy=0
    values=df[cl].unique()
    for value in values:
        fraction=df[cl].value_counts()[value]/len(df[cl])
        entropy += -fraction*np.log2(fraction)
    return entropy


In [4]:
def find_entropy_attribute(df,attribute):
    cl=df.keys()[-1]
    target=df[cl].unique()
    var=df[attribute].unique()
    ent2=0
    for vb in var:
        et=0
        for tv in target:
            num=len(df[attribute][df[attribute]==vb][df[cl]==tv])
            den=len(df[attribute][df[attribute]==vb])
            fraction=num/(den+eps)
            et+=-fraction*log(fraction+eps)
        fraction2=den/len(df)
        ent2+=-fraction2*et
    return abs(ent2)


In [5]:
def find_winner(df):
    ig=[]
    for key in df.keys()[:-1]:
        ig.append(find_entropy(df)-find_entropy_attribute(df,key))
    return df.keys()[:-1][np.argmax(ig)]


In [6]:
def get_Subtable(df,node,value):
    return df[df[node]==value].reset_index(drop=True)

In [7]:
def buildTree(df,tree=None):
    cl=df.keys()[-1]
    node=find_winner(df)
    atvl=np.unique(df[node])
    if tree is None:
        tree={}
        tree[node]={}
    for vl in atvl:
        subtable=get_Subtable(df,node,vl)
        clvl,count=np.unique(subtable['play'],return_counts=True)
        if len(count)==1:
            tree[node][vl]=clvl[0]
        else:
            tree[node][vl]=buildTree(subtable)
    return tree



In [9]:

df=pd.read_csv("id31.csv")
tree=buildTree(df)
pprint.pprint(tree)

{'Outlook': {'Overcast': 'Yes',
             'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}},
             'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}
